# Line Following using OpenCV 
**Full Coding available at [Line_Following.py](Line_Following.py)** 
<br>
This is the documentation for line following using OpenCV 

***Libraries Used In The Script***
- Picamera2
- OpenCV
- NumPy
- Time 
- RPi_Robot_Hat_Lib


## Installing Libraries @ Dependencies 
- Picamera2 
- libcamera
- os
- time 
- OpenCV 
- numpy 
- RPi_Robot_Hat_Lib 

### First, let's install the some libraries ! 
1. OpenCV 
    - Follow the link [OpenCV.org](https://docs.opencv.org/4.x/d2/de6/tutorial_py_setup_in_ubuntu.html)
2. NumPy 
    - Follow the link [numpy.org](https://numpy.org/install/)


### Verify the installed libraries 
1. OpenCV 
    - In Python File type: <br>
    `import cv2` <br>
    `print(cv2.__version__)`
    - The coding above will print the version of opencv library installed  
2. NumPy 
    - In Python File type: <br>
    `import numpy as np`
    - Installation sucess if codeing run without error 

## Let's Start Coding ! 
### 1. Import all the libraries 
- OpenCV 
- NumPy 
- Picamera2 
- libcamera
- RPi_Robot_Hat_Lib

In [ ]:
from picamera2 import Picamera2
from libcamera import controls
import cv2
import numpy as np
import time
from RPi_Robot_Hat_Lib import RobotController


### 2. The `init` function
- Initialise Libraries 
    - RPi_Robot_Hat_Lib (To control the movement of the robot)
    - Picamera2 (To access the camera)
    - libcamera (To control the camera) 

- Configure the camera  
    ```py 
        picam2 = Picamera2() 
        picam2.preview_configuration.main.size = (640, 480) # Configure the video size for the camera 
        picam2.preview_configuration.main.format = 'RGB888' # Configure the video format 
        picam2.preview_configuration.align() 
        picam2.configure("preview")
        picam2.start()  # Start camera preview
        ## Set the camera to Continous Auto Focus 
        picam2.set_controls({"AfMode": controls.AfModeEnum.Continuous}) # Set the AutoFocus Mode to continuous mode
    ```
    ***Note**: the `picam2.set_controls({"AfMode": controls.AfModeEnum.Continuous})` is only avaialibe for Raspberry Camera Module 3 and above only. *Comment* out the code by adding **#** infront. 

- Set the position of the camera 
    - Control the servo position by using the RobotControler class from RPi_Robot_Hat_Lib. 
    ```py
        vertical = 1 # Assign 1 as servo channel for vertical movement 
        horizontal = 2 # Assign 1 as servo channel for horizontal movement 
        Motor.set_servo(vertical, 180) # using the set_servo() function to set the the vertical position 
        Motor.set_servo(horizontal, 90) # using the set_servo() function to set the the horizontal position 
    ```





In [ ]:
def init():
    """
    Initialize the Motor, Encoder, and camera preview.

    Returns
    -------
    picamera2.Picamera2
        The initialized Picamera2 object.
    """
    global picam2, horizontal, vertical, frame_center, Motor, enc
    # Initialise the Motor
    Motor = RobotController()
    


    # Set PanTilt and servo HAT
    vertical = 1
    horizontal = 2
    Motor.set_servo(vertical, 180)
    Motor.set_servo(horizontal, 90)

    # Set preview configuration (modify resolution as needed)
    picam2 = Picamera2()
    picam2.preview_configuration.main.size = (640, 480)
    picam2.preview_configuration.main.format = 'RGB888'
    picam2.preview_configuration.align()
    picam2.configure("preview")
    picam2.start()  # Start camera preview
    ## Continous Auto Focus 
    picam2.set_controls({"AfMode": controls.AfModeEnum.Continuous})

    frame_center = (picam2.preview_configuration.main.size[0] // 2 ,
                    picam2.preview_configuration.main.size[1] // 2)


    return picam2


### 3. Main function for Line Following 
- Start Init function 
    ```py
    init()
    ``` 
- While True Loop 
    - Obtain the captured video array and store in to frame variable 
         ```py
         frame = Picam2.capture_array()
         ```
     - Set the value for Upper Black Threshold and Lower Black Threshold
        ```py
        Black_lower = np.array([0,0,0], dtype = "uint8")
        Black_upper = np.array([179, 255, 40], dtype = "uint8")
        Blacklines = cv2.inRange(frame, Black_lower, Black_upper)
        ```
    - Pass the threshold to an Edge Detector 
        ```py
        canny_edges = cv2.Canny(Blacklines, 50, 150)
        ```
    - Obtain the contour (Detected data) and anotate it on the frame
        ```py
        contours, _  = cv2.findContours(Blacklines, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
          if len(contours) > 0 :
              for contour in contours:
                  cv2.drawContours(frame, [contour] , 0 , (0,255,0), 2)
        ```
    - Look for the largest contour and drawit out
        ```py
        if len(contours) > 0:
                largeset = max(contours, key = cv2.contourArea)
                cv2.drawContours(frame, largeset , 0 , (0,0,255), 5)
        ```
    - Calculate the center point of the largest contour 
        ```py
        M = cv2.moments(largeset)
               cx = int(M["m10"] / M["m00"]) if M["m00"] != 0 else 0
               cy = int(M["m01"] / M["m00"]) if M["m00"] != 0 else 0
               cv2.putText(frame, f" Offset: X = {cx}, Y: = {cy}", (10,20), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0,255,255), 2 )
        ``` 
    - Based on the calculated pcenter point start the movement  
        ```py
        if cx >= 350:
            print("Going Right")
            Motor.move(speed=0, turn=-15)
        
        if cx < 350 and cx > 300:
            print("On track")
            Motor.Forward(20)
        if cx < 300:
            print("Going Left")
            Motor.move(speed=0, turn=-15)
        ```
    - Draw a circle on the center point of the contour for better visualisation
        ```py
        cv2.circle(frame, (cx,cy),5,(255,255,0),-1) 
        ```
    - Brake when the black contour is not detected 
        ```py
        else:
           Motor.Brake()
           print("Nothing Detected")
        ```
    - Show the frame for Black detection, edge detection and the main frame 
        ```py
        cv2.imshow("Black Lines", Blacklines)
        cv2.imshow("Edge Detection", canny_edges)
        cv2.imshow("Main", frame) 
    
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        ```



In [ ]:
def main():
    """
    main fucntion is to perform object tracking and motor control 
    """
    global picam2, frame_center, Motor

    init()

    while True:
        frame = picam2.capture_array()
        
        
        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)
        Black_lower = np.array([56,22,27], dtype = "uint8")
        Black_upper = np.array([179, 255, 255], dtype = "uint8")
        Blacklines = cv2.inRange(frame, Black_lower, Black_upper)
               
                
        canny_edges = cv2.Canny(Blacklines, 50, 150)
        
        contours, _  = cv2.findContours(Blacklines, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if len(contours) > 0 :
            for contour in contours:
                cv2.drawContours(frame, [contour] , 0 , (0,255,0), 2)
                
            
        frame_offset = (0,0)
        if len(contours) > 0:
            largeset = max(contours, key = cv2.contourArea)
            cv2.drawContours(frame, largeset , 0 , (0,0,255), 5)
            
            M = cv2.moments(largeset)
            cx = int(M["m10"] / M["m00"]) if M["m00"] != 0 else 0
            cy = int(M["m01"] / M["m00"]) if M["m00"] != 0 else 0
            cv2.putText(frame, f" Offset: X = {cx}, Y: = {cy}",
                        (10,20), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0,255,255), 2 )
            
            if cx >= 350:
                print("Going Left")
                Motor.move(speed=0, turn=-15)
            
            if cx < 350 and cx > 300:
                print("On track")
                Motor.Forward(20)
            if cx < 300:
                print("Going Right")
                Motor.move(speed =0 , turn=15)

            
            cv2.circle(frame, (cx,cy),5,(255,255,0),-1)
        else:
            Motor.Brake()
            print("Nothing Detected")


        cv2.imshow("Black Lines", Blacklines)
        cv2.imshow("Edge Detection", canny_edges)
        cv2.imshow("Main", frame) 
    
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break 


### 4. Wraping up 
- when the programe start run the main function 
    ```py
    try:
        if __name__ == '__main__': 
            main() # When the program starts, run the main() funstion 
    ``` 
- When there is keyboard interupt (Ctrl + c)
    ```py
    except KeyboardInterrupt:
        Motor.Brake() # Stop the motor (If the motor is running)
        Motor.cleanup() # Reset all the Hardware configuration, Reset the position of the Pan Tilt Hat. 
    ```


    

In [ ]:
try:
    if __name__ == '__main__': 
        main()
        
except KeyboardInterrupt:
    Motor.Brake()
    Motor.cleanup()
    